In [ ]:
!git clone https://github.com/unfiltered-syrup/TECH-UB.0024_Final
!pip install spotipy --upgrade

In [ ]:
!pip install flask-ngrok > /dev/null
!pip install -q pyngrok
!ngrok authtoken 23FD4t7XWxBjf9HXFzSOjo7kgbG_5jnc58AtdCLCkr8J6nh3b # <-- replace with your own

In [ ]:
#imports and downloads
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request as flask_request
from flask import Flask, redirect, url_for, render_template
import sys
import json
import requests
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier
import nltk
nltk.download('punkt')
import pandas as pd
import re
import spacy
emotion = ''
with open('/content/TECH-UB.0024_Final/TECH-Final-Frontend/emotion_complete.csv', encoding="ISO-8859-1") as fp:
  cl = NaiveBayesClassifier(fp, format="csv")
app = Flask(__name__, template_folder='/content/TECH-UB.0024_Final/TECH-Final-Frontend/', static_folder='/content/TECH-UB.0024_Final/TECH-Final-Frontend/')
run_with_ngrok(app)
def before_request():
    app.jinja_env.cache = {}
@app.route("/")
def testing():
    return render_template("front_page.html")

def split(vent):
    return(re.split('[?.!]', vent))
@app.route('/get_strings', methods=['POST'])
def get_string():
  global tracks
  alphabets= "([A-Za-z])"
  prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
  suffixes = "(Inc|Ltd|Jr|Sr|Co)"
  starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
  acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
  websites = "[.](com|net|org|io|gov)"

  v2 = pd.read_excel(open('/content/TECH-UB.0024_Final/vent_library.xlsx', 'rb'),sheet_name='vent_library')
  print(v2)
  happy_df = v2[v2['Mood'] == 'Happy']
  angry_df = v2[v2['Mood'] == 'Angry']
  sad_df = v2[v2['Mood'] == 'Sad']
  heartbreak_df = v2[v2['Mood'] == 'Heartbreak']
  focus_df = v2[v2['Mood'] == 'Focus']
  selflove_df = v2[v2['Mood'] == 'Self Love']
  inspiration_df = v2[v2['Mood'] == 'Inspiration']
  party_df = v2[v2['Mood'] == 'Party']
  love_df = v2[v2['Mood'] == 'Love']
  friendship_df = v2[v2['Mood'] == 'Friendship']
  happy_songs = happy_df[['Track']]
  angry_songs = angry_df[['Track']]
  sad_songs = sad_df[['Track']]
  heartbreak_songs = heartbreak_df[['Track']]
  focus_songs = focus_df[['Track']]
  selflove_songs = selflove_df[['Track']]
  inspiration_songs = inspiration_df[['Track']]
  party_songs = party_df[['Track']]
  love_songs = love_df[['Track']]
  friendship_songs = friendship_df[['Track']]
  emotion_list=[]
  import string
  from string import punctuation
  text = flask_request.form["text_input"]
  if text[len(text)-1] in string.punctuation:
    text = text
  else:
    text = text+'.'
  print(text)
  text = " " + text + "  "
  text = text.replace("\n"," ")
  text = re.sub(prefixes,"\\1<prd>",text)
  text = re.sub(websites,"<prd>\\1",text)
  if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
  text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
  text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
  text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
  text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
  text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
  text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
  text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
  if "”" in text: text = text.replace(".”","”.")
  if "\"" in text: text = text.replace(".\"","\".")
  if "!" in text: text = text.replace("!\"","\"!")
  if "?" in text: text = text.replace("?\"","\"?")
  text = text.replace(".",".<stop>")
  text = text.replace("?","?<stop>")
  text = text.replace("!","!<stop>")
  text = text.replace("<prd>",".")
  sentences = text.split("<stop>")
  sentences = sentences[:-1]
  sentences = [s.strip() for s in sentences]
  tracks = []
  for sent in sentences:
    blob=TextBlob(sent,classifier=cl)
    emotion_list.append(blob.classify())
  for x in emotion_list:
        if x == 'happiness':
            tracks+=happy_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'anger':
            tracks+=angry_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'sadness':
            tracks+=sad_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'heartbroken':
            tracks+=heartbreak_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'working':
            tracks+=focus_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'worry':
            tracks+=selflove_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'boredom':
            tracks+=inspiration_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'fun':
            tracks+=party_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'love':
            tracks+=love_songs.sample(30//len(emotion_list))['Track'].to_list()
        if x == 'lonely':
            tracks+=friendship_songs.sample(30//len(emotion_list))['Track'].to_list()
  return render_template("front_page2.html", param1 = json.dumps(tracks))
  
@app.route('/return_result')
def results(): #the following is called after runtime, but doesn't support collab notebook
    access_token="BQD7MJWQHAz-0a4IKKO9LqYrFKOOB8PcPah_b5Em8jVpKETuyiWUWo1tSslLYg65VozKe_hEdRGfAo3iH84"
    # base URL of all Spotify API endpoints
    headers = {
        'Authorization': 'Bearer {token}'.format(token=access_token)
    }
    BASE_URL = 'https://api.spotify.com/v1/'
    id_list=[]
    for song in tracks:
        r = requests.get(BASE_URL + 'search?' + "q=" + str(song), "type=track", headers=headers)
        r = r.json()
        if len(r["tracks"]["items"])==0:
            pass
        else:
            id_list.append(r["tracks"]["items"][0]["id"])
    client_ID = 'd39c2f2f44284bda99c32b8c897f5f04'
    client_SECRET = 'f332a98be03143cdad37f764d90fddf0'
    redirect_url = "http://localhost:8888/callback/"
    import spotipy
    from spotipy.oauth2 import SpotifyOAuth
    scope = "playlist-modify-public"
    spot = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_ID, client_secret=client_SECRET,
                                                     redirect_uri=redirect_url, scope=scope))
    results = spot.user_playlist_create(user="sakhikamal", name="New Vent", public=True, collaborative=False)
    spot.playlist_add_items(results["id"], id_list, position=None) 
    return 'nothing' 

if __name__ == '__main__':
  app.jinja_env.auto_reload = True
  app.config['TEMPLATES_AUTO_RELOAD'] = True
  app.run()

In [ ]:
!mkdir templates/

In [ ]:
!mv TECH-UB.0024_Final/font-end/* templates